# Initial Steps in Image Recognition

**Students**


**Problem setup**

In this assignment you are asked to apply ANNs in order to classify a given silhouette (image) of a vehicle as one of four types
-	a double decker bus (class 0),
-	Opel Manta 400 (class 1),
-	Saab 9000 (class 2), 
-	Cheverolet van (class 3),

using a set of features extracted from the silhouette. The vehicle may be viewed from one of many different angles. The dataset is given in two files: **veh_train.csv** and **veh_test.csv**. Variable class is the outcome (nominal) variable.

One of the principal questions of this assignment is whether PCA can help (or, at least, not make worse) predicting classes.

Data source and detailed description: https://archive.ics.uci.edu/ml/datasets/Statlog+%28Vehicle+Silhouettes%29 



In [1]:
# load traditional libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA as skPCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
# load KERAS
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
# set constants
c_nclass = 4

list_feats = ["V1", "V2", "V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16",
                             "V17","V18"]

In [4]:
# define a function for your ANN
def myANN(input_dim):
    
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(c_nclass, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    
    return model


### Reading and preparing the data

In [5]:
# read the data
veh_train = pd.read_csv('veh_train.csv', header=0)
veh_test = pd.read_csv('veh_test.csv', header=0)

In [6]:
# construct the training and the test samples
X_train = veh_train.drop('class', axis=1)
y_train = veh_train['class']

X_test = veh_test.drop('class', axis=1)
y_test = veh_test['class']

In [7]:
# since PCA will be applied, let's scale the features

# scale the training data
X_train_sc = StandardScaler().fit_transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc)
X_train_sc.columns = list_feats

# scale the test data
X_test_sc = StandardScaler().fit_transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc)
X_test_sc.columns = list_feats

# preview the scaled data
X_train_sc.head(6)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18
0,0.142657,0.493358,0.048570,0.266144,1.350583,0.331154,-0.211507,0.142691,-0.233227,0.751569,-0.406215,-0.350463,0.268827,-0.318038,-0.084475,0.389751,-0.334073,0.170051
1,1.241760,0.817326,1.519254,1.204229,0.563227,0.331154,1.140490,-1.135304,0.923806,0.682146,1.099655,1.094856,1.360598,0.096104,1.551923,-0.418356,-0.172632,0.036608
2,-0.101589,-0.640530,-0.015373,-0.308812,0.169549,0.103675,-0.752306,0.653888,-0.618905,-0.359197,-0.918852,-0.745667,-1.459809,-1.284369,-0.084475,-0.302912,1.603217,1.504479
3,-1.078570,-0.154578,-0.782686,1.083186,5.418592,9.885314,-0.602084,0.526089,-0.618905,-0.289774,1.676371,-0.655335,0.390135,7.550654,0.529174,-0.187469,-1.464159,-1.698148
4,1.608128,1.951214,1.519254,0.084579,-1.536390,-0.578765,2.582619,-1.902101,2.852195,1.445798,2.925923,2.912796,2.694984,1.752670,-0.289024,-0.418356,-1.302718,-1.698148
5,-0.467957,-0.316562,-1.038457,-0.369334,0.432001,0.103675,-0.962616,0.909487,-1.004582,-0.150928,-0.854772,-0.903749,-0.337712,-0.732180,-0.698124,-1.111019,0.634572,0.837265


## Task 1 (1 point). Descriptive analysis of the ORIGINAL training sample

Read the description of the features. Analyze the structure and features of the dataset (no graphs!). Do the following on the **training sample**:

-	Check whether the classes are balanced.
-	Calculate summary statistics and correlations of the ORIGINAL features.
-	Do you expect that PCA may be helpful? Give brief comments.


In [10]:
# check the balance of classes
veh_train['class'].value_counts(normalize=True)

0    0.264012
2    0.256637
1    0.243363
3    0.235988
Name: class, dtype: float64

In [11]:
veh_test['class'].value_counts(normalize=True)

1    0.279762
2    0.255952
3    0.232143
0    0.232143
Name: class, dtype: float64

We can see that in both samplings - train and test, the classes are balanced

In [12]:
# summary statistics on features
veh_train.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,class
count,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000
mean,93.831858,44.954277,82.240413,169.205015,61.707965,8.544248,169.039823,40.883481,20.604720,148.174041,188.678466,441.075221,175.135693,72.303835,6.412979,12.623894,189.069322,195.725664,1.464602
std,8.194536,6.178007,15.650532,33.070412,7.626063,4.399237,33.308680,7.830535,2.594752,14.415109,31.234235,177.254322,32.998311,7.249246,4.892397,8.668615,6.198792,7.499382,1.118298
min,73.000000,33.000000,40.000000,105.000000,47.000000,2.000000,112.000000,26.000000,17.000000,119.000000,130.000000,184.000000,109.000000,59.000000,0.000000,0.000000,176.000000,181.000000,0.000000
25%,88.000000,40.000000,70.000000,141.000000,57.000000,7.000000,147.000000,33.000000,19.000000,137.000000,167.250000,320.000000,149.000000,67.000000,2.000000,6.000000,185.000000,190.250000,0.000000
50%,93.000000,44.000000,80.000000,168.000000,61.000000,8.000000,157.000000,43.000000,20.000000,146.000000,179.000000,365.500000,174.000000,71.000000,6.000000,11.000000,188.000000,197.000000,1.000000
75%,100.000000,49.000000,98.000000,195.750000,65.000000,10.000000,198.000000,46.000000,23.000000,159.000000,217.000000,586.750000,198.750000,75.000000,9.000000,18.000000,193.000000,201.000000,2.000000
max,117.000000,59.000000,110.000000,322.000000,133.000000,52.000000,265.000000,61.000000,29.000000,188.000000,288.000000,1018.000000,268.000000,127.000000,22.000000,41.000000,206.000000,211.000000,3.000000


In [14]:
#correlation matrix
pd.DataFrame.corr(X_train, method="pearson").round(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18
V1,1.00,0.69,0.79,0.70,0.10,0.15,0.81,-0.79,0.81,0.67,0.77,0.82,0.59,-0.25,0.23,0.15,0.29,0.35
V2,0.69,1.00,0.79,0.63,0.15,0.25,0.86,-0.83,0.86,0.96,0.82,0.85,0.94,0.08,0.15,-0.04,-0.14,0.01
V3,0.79,0.79,1.00,0.78,0.16,0.27,0.91,-0.91,0.89,0.77,0.87,0.89,0.70,-0.22,0.12,0.25,0.13,0.31
V4,0.70,0.63,0.78,1.00,0.65,0.43,0.75,-0.80,0.72,0.57,0.79,0.73,0.55,-0.21,0.05,0.18,0.37,0.46
V5,0.10,0.15,0.16,0.65,1.00,0.63,0.11,-0.19,0.09,0.13,0.24,0.09,0.12,0.11,-0.08,-0.02,0.23,0.26
V6,0.15,0.25,0.27,0.43,0.63,1.00,0.17,-0.19,0.17,0.32,0.29,0.15,0.20,0.27,-0.00,0.05,-0.05,0.13
V7,0.81,0.86,0.91,0.75,0.11,0.17,1.00,-0.97,0.99,0.81,0.96,1.00,0.80,-0.00,0.07,0.19,-0.02,0.09
V8,-0.79,-0.83,-0.91,-0.80,-0.19,-0.19,-0.97,1.00,-0.95,-0.77,-0.95,-0.96,-0.77,0.08,-0.05,-0.17,-0.09,-0.19
V9,0.81,0.86,0.89,0.72,0.09,0.17,0.99,-0.95,1.00,0.81,0.95,0.99,0.80,0.01,0.08,0.19,-0.04,0.07
V10,0.67,0.96,0.77,0.57,0.13,0.32,0.81,-0.77,0.81,1.00,0.75,0.79,0.86,0.06,0.14,-0.03,-0.13,0.05


**Give your comments here**

## Task 2 (1 point). ANN on the SCALED full dataset

Estimate and validate a simple 1-layer ANN on the SCALED full data. Do the following:

- Transform the outcome variable on both sample appropriately.
- Run the model (use 100 epochs).
- Validate its performance.
- Give brief comments.



In [18]:
# prepare the outcome variable
Y_01_train = np_utils.to_categorical(y_train, c_nclass)
Y_01_test = np_utils.to_categorical(y_test, c_nclass)

In [20]:
# setup the Model1 with the myANN function
model1 = myANN(18)

In [24]:
# train Model1
np.random.seed(12345) # do not change it!
model1.fit(X_train_sc, Y_01_train, epochs=100, validation_data = (X_test_sc, Y_01_test))

Train on 678 samples, validate on 168 samples
Epoch 1/100
678/678 [==============================] - 1s 1ms/step - loss: 1.3141 - accuracy: 0.3717 - val_loss: 1.2480 - val_accuracy: 0.4821
Epoch 2/100
678/678 [==============================] - 0s 89us/step - loss: 1.0855 - accuracy: 0.5501 - val_loss: 1.1148 - val_accuracy: 0.5655
Epoch 3/100
678/678 [==============================] - 0s 104us/step - loss: 0.9558 - accuracy: 0.6622 - val_loss: 1.0183 - val_accuracy: 0.5952
Epoch 4/100
678/678 [==============================] - 0s 105us/step - loss: 0.8654 - accuracy: 0.7035 - val_loss: 0.9610 - val_accuracy: 0.6250
Epoch 5/100
678/678 [==============================] - 0s 108us/step - loss: 0.7930 - accuracy: 0.7286 - val_loss: 0.8912 - val_accuracy: 0.6905
Epoch 6/100
678/678 [==============================] - 0s 98us/step - loss: 0.7285 - accuracy: 0.7581 - val_loss: 0.8448 - val_accuracy: 0.6964
Epoch 7/100
678/678 [==============================] - 0s 89us/step - loss: 0.6765 - acc

678/678 [==============================] - 0s 89us/step - loss: 0.2649 - accuracy: 0.8997 - val_loss: 0.4913 - val_accuracy: 0.7857
Epoch 58/100
678/678 [==============================] - 0s 83us/step - loss: 0.2616 - accuracy: 0.9027 - val_loss: 0.4916 - val_accuracy: 0.7857
Epoch 59/100
678/678 [==============================] - 0s 79us/step - loss: 0.2560 - accuracy: 0.9041 - val_loss: 0.5023 - val_accuracy: 0.7798
Epoch 60/100
678/678 [==============================] - 0s 86us/step - loss: 0.2521 - accuracy: 0.9130 - val_loss: 0.5110 - val_accuracy: 0.7857
Epoch 61/100
678/678 [==============================] - 0s 78us/step - loss: 0.2548 - accuracy: 0.8982 - val_loss: 0.4851 - val_accuracy: 0.7917
Epoch 62/100
678/678 [==============================] - 0s 79us/step - loss: 0.2548 - accuracy: 0.9100 - val_loss: 0.4963 - val_accuracy: 0.7738
Epoch 63/100
678/678 [==============================] - 0s 78us/step - loss: 0.2455 - accuracy: 0.9145 - val_loss: 0.4955 - val_accuracy: 0.785

In [27]:
Y_01_test_pred = model1.predict(X_test_sc)

Y_01_test_pred[1]

array([9.7514957e-01, 2.0808069e-02, 3.6627930e-03, 3.7960117e-04],
      dtype=float32)

In [26]:
# prediction of classes

Y_01_test_pred = model1.predict(X_test_sc)

Y_01_test_pred[1]
# classification report
print(classification_report(y_true=Y_01_test, y_pred=Y_01_test_pred.round(0)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        39
           1       0.68      0.57      0.62        47
           2       0.64      0.65      0.64        43
           3       0.88      0.95      0.91        39

   micro avg       0.79      0.77      0.78       168
   macro avg       0.79      0.79      0.79       168
weighted avg       0.78      0.77      0.78       168
 samples avg       0.77      0.77      0.77       168



//anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Give your comments here**

## Task 3 (2 points). PCA analysis

- Run a PCA analysis on the scaled features.
- Calculate variances and cumulative variances.
- For further analysis, leave only first 9 principal components. How much variance of the original dataset do they explain?
- Give brief comments.

In [28]:
# run a PCA
pca = skPCA(n_components=None)
pca.fit(X_train_sc)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [29]:
# % of variance explained by PCs
var_exp = pca.explained_variance_ratio_
var_exp

array([5.23679614e-01, 1.72827778e-01, 1.01457506e-01, 6.67645018e-02,
       4.89127117e-02, 2.97375344e-02, 1.98983675e-02, 1.22139920e-02,
       8.82540635e-03, 5.09920830e-03, 3.22658195e-03, 2.42373565e-03,
       1.95514390e-03, 1.07875398e-03, 8.29131024e-04, 7.09152954e-04,
       3.41093682e-04, 1.97865794e-05])

In [30]:
# cumulative variance
cum_var_exp = np.cumsum(var_exp)
cum_var_exp

array([0.52367961, 0.69650739, 0.7979649 , 0.8647294 , 0.91364211,
       0.94337965, 0.96327801, 0.97549201, 0.98431741, 0.98941662,
       0.9926432 , 0.99506694, 0.99702208, 0.99810084, 0.99892997,
       0.99963912, 0.99998021, 1.        ])

In [32]:
# make first 9 components - for the training and test samples
PC1_loadings = pca.components_[0] 
#for the first component

PC2_loadings = pca.components_[1] 
PC3_loadings = pca.components_[2] 
PC4_loadings = pca.components_[3] 
PC5_loadings = pca.components_[4] 
PC6_loadings = pca.components_[5] 
PC7_loadings = pca.components_[6] 
PC8_loadings = pca.components_[7] 
PC9_loadings = pca.components_[8] 

In [33]:
PC1_values = pca.transform(X_train_sc)[:,0]
#taking the first column out of we got
PC2_values = pca.transform(X_train_sc)[:,1]
PC3_values = pca.transform(X_train_sc)[:,2]
PC4_values = pca.transform(X_train_sc)[:,3]
PC5_values = pca.transform(X_train_sc)[:,4]
PC6_values = pca.transform(X_train_sc)[:,5]
PC7_values = pca.transform(X_train_sc)[:,6]
PC8_values = pca.transform(X_train_sc)[:,7]
PC9_values = pca.transform(X_train_sc)[:,8]

In [38]:
PC1_values = pca.transform(X_train_sc)[:,0]
#taking the first column out of we got
PC2_values = pca.transform(X_train_sc)[:,1]
PC3_values = pca.transform(X_train_sc)[:,2]
PC4_values = pca.transform(X_train_sc)[:,3]
PC5_values = pca.transform(X_train_sc)[:,4]
PC6_values = pca.transform(X_train_sc)[:,5]
PC7_values = pca.transform(X_train_sc)[:,6]
PC8_values = pca.transform(X_train_sc)[:,7]
PC9_values = pca.transform(X_train_sc)[:,8]

array([ 0.29880345,  3.74437669, -1.81719983,  0.6602405 ,  6.42285853,
       -2.19403958, -4.54912514, -3.5591212 , -0.86214409, -1.44927976,
       -5.29704398,  0.29854454,  4.14919613,  4.21121305, -1.39708005,
       -5.10742168, -1.2257356 , -2.80504379,  3.83113545, -4.83204158,
        4.56827006,  1.51589253, -3.22483961, -0.91641204, -2.09614437,
        3.86839488, -1.24747588, -2.24258934,  3.15538668,  4.00617063,
        2.95053715, -5.33842343, -0.2325941 , -2.80650546, -0.21365254,
       -3.86960044,  4.51306847, -1.1963665 ,  6.04490363, -3.18832071,
        1.16971839, -0.85383792, -4.04562329, -2.18062866,  1.10980907,
       -2.23684527, -2.40816134,  4.90266082,  2.26316907,  3.59064047,
        5.47418032,  0.04618475, -5.28195135, -1.21020777,  3.00073697,
       -0.66239272, -2.67991669, -2.14694032, -1.38986924,  3.36522808,
       -3.89013461, -1.57235526,  6.79012108, -1.61696294, -4.90249709,
       -4.98898884,  4.93997331, -0.87333615,  3.32523504,  0.04